# IMDB 영화 리뷰 감성분석

In [1]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving labeledTrainData.tsv to labeledTrainData.tsv


- zip 파일을 업로드 하면 
- !unzip filename

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv(filename, sep='\t', quoting=3) # quoting = 3으로 설정해주면 인용구(따옴표)를 무시
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [15]:
df.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [6]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

## 1. 전처리

In [8]:
# <br /> 태그는 공백으로 변환
df['review'] = df.review.str.replace('<br />', ' ')

In [13]:
# 구둣점, 숫자 제거 --> 영어 이외의 문자는 공백으로 변환
import re
df['review'] = df.review.apply(lambda x: re.sub('[^A-Za-z]', ' ', x))

In [14]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

## 2. 학습 및 테스트 데이터 셋으로 분리

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, test_size=0.2,       # df.review, df.sentiment : X, y, 시리즈, 넘파이 전달 가능
    stratify=df.sentiment, random_state=2021    # Stratified sampling는 훈련데이터를 나눌 때 무작위로 샘플링을 하되, original dataset의 클래스 비율이 train, test set에서도 동일하게 유지되는 것을 보장
)                                                # 200000개 - 훈령용, 5000개 - 테스트용
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000,), (5000,), (20000,), (5000,))

## 3. 텍스트 변환

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

- CountVectorizerm ngram_range=(1,1)

In [23]:
# 이렇게 변환하면 안됨!!! fit_transform
cvect1 = CountVectorizer(stop_words='english')
X_train_cv1 = cvect1.fit_transform(X_train)
X_test_cv1 = cvect1.fit_transform(X_test)

In [25]:
X_train_cv1.shape, X_test_cv1.shape  # train과 text의 단어 개수가 다름! 같은 단어지만 다른 숫자를 갖게 됨!(문제)

((20000, 66971), (5000, 37187))

In [27]:
# X_train으로 fit을 먼저한 뒤 변환해야 함!
# train, test data set에 있는 동일한 단어에 대해서 동일한 값으로 변환하기 위해
# 반드시 아래와 같이 해야함!
cvect1 = CountVectorizer(stop_words='english')
cvect1.fit(X_train)
X_train_cv1 = cvect1.transform(X_train)
X_test_cv1 = cvect1.transform(X_test)
X_train_cv1.shape, X_test_cv1.shape  # 단어의 개수가 같음 => 같은 숫자가 같은 단어를 가리킴!

((20000, 66971), (5000, 66971))

- CountVectorizer, ngram_range(1, 2)

In [42]:
cvect2 = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape

((20000, 1461350), (5000, 1461350))

- TfidVectorizer, ngram_range(1,1)

In [47]:
tvect1 = TfidfVectorizer(stop_words='english')
tvect1.fit(X_train)
X_train_tv1 = tvect1.transform(X_train)
X_test_tv1 = tvect1.transform(X_test)
X_train_tv1.shape, X_test_tv1.shape

((20000, 66971), (5000, 66971))

- TfidVectorizer, ngram_range(1,2)

In [52]:
tvect2 = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tvect2.fit(X_train)
X_train_tv2 = tvect2.transform(X_train)
X_test_tv2 = tvect2.transform(X_test)
X_train_tv2.shape, X_test_tv2.shape

((20000, 1461350), (5000, 1461350))

## 4. 로지스틱 회귀 모델로 학습(훈련)

In [28]:
# 시그모이드 함수 사용
from sklearn.linear_model import LogisticRegression

In [30]:
# 객체 생성 및 훈련 - 훈련 데이터의 X와 y 모두 필요
lr1 = LogisticRegression(max_iter=500)
%time lr1.fit(X_train_cv1, y_train) # 학습, %time: 시간 측정

CPU times: user 6.31 s, sys: 7.12 s, total: 13.4 s
Wall time: 7.91 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
# 객체 생성 및 훈련 - 훈련 데이터의 X와 y 모두 필요
lr2 = LogisticRegression(max_iter=500)
%time lr2.fit(X_train_cv2, y_train) # 학습, %time: 시간 측정

CPU times: user 47.2 s, sys: 33.2 s, total: 1min 20s
Wall time: 49.2 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
lr3 = LogisticRegression(max_iter=500)
%time lr3.fit(X_train_tv1, y_train)

CPU times: user 1.71 s, sys: 1.65 s, total: 3.36 s
Wall time: 1.76 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
lr4 = LogisticRegression(max_iter=500)
%time lr4.fit(X_train_tv2, y_train)

CPU times: user 14.1 s, sys: 10.3 s, total: 24.4 s
Wall time: 15 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
# 테스트 데이터에 대해 예측 - 테스트 데이터의 X만 필요
pred1 = lr1.predict(X_test_cv1) 

In [45]:
pred2 = lr2.predict(X_test_cv2) 

In [49]:
pred3 = lr3.predict(X_test_tv1) 

In [54]:
pred4 = lr4.predict(X_test_tv2) 

## 5. 평가

In [32]:
from sklearn.metrics import accuracy_score

In [33]:
score1 = accuracy_score(y_test, pred1) # 정답값, 예측값
score1

0.8832

In [46]:
score2 = accuracy_score(y_test, pred2)
score2

0.8914

In [50]:
score3 = accuracy_score(y_test, pred3) # 상황마다 Countervectorizer또는 TfidVectorizer 정확도는 달라짐.(일반적 X)
score3

0.893

In [55]:
score4 = accuracy_score(y_test, pred4) 
score4

0.883

## 6. 비지도 학습 기반 감성분석
- 감성어휘 사전

In [57]:
import nltk
nltk.download('vader_lexicon') # 단어사전

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [58]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sanalyzer = SentimentIntensityAnalyzer()

In [59]:
senti_score = sanalyzer.polarity_scores(df.review[0]) 
# polarity_scores : 긍정, 부정, 중립 감성 지수를 구하고 이 3가지 지수를 적절히 조합해 계산한 총 감성 지수까지 계산
# 총 감성 지수는 -1에서 1사이의 값으로 표현된다.(총 감성 지수는 compound라고 되어 있는 key값의 value이다.
senti_score

{'compound': -0.7943, 'neg': 0.13, 'neu': 0.743, 'pos': 0.127}

In [60]:
df.sentiment[0]

1

In [63]:
# compound score 값이 0.1보다 크면 긍정(1), 낮으면 부정(0)
# compound(총 감성 지수) 값이 사전에 설정한 임계값을 기준으로 임계값보다 높으면 긍정 감성, 낮으면 부정감성으로 판단. vader의 보통 임계값은 0.1로 설정
def get_sentiment(sabakyzer, review, threshold): # threshold : 임계값
    score = sanalyzer.polarity_scores(review)
    compound_score = score['compound']
    final_score = 1 if compound_score >= threshold else 0
    return final_score

In [64]:
df['vader_pred'] = df.review.apply(lambda x: get_sentiment(sanalyzer, x, 0.1))

In [67]:
# 평가, real y - df.sentiment, 예측 y - df.vader_pred
accuracy_score(df.sentiment, df.vader_pred)

0.69476

In [68]:
for threshold in [0, 0.1, 0.2, 0.3]:
    df['vader_pred'] = df.review.apply(lambda x: get_sentiment(sanalyzer, x, threshold))
    score = accuracy_score(df.sentiment, df.vader_pred)
    print(f'threshold값 {threshold} : {score}')

0.69012
0.69476
0.69868
0.7022
